<a href="https://colab.research.google.com/github/jorgealarconlucas/TFG/blob/master/Modelos_de_ML_con_tendencia_temporal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Machine learning añadiendo información sobre la tendencia temporal

In [1]:
import pandas as pd
import numpy as np
import statistics as stats
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import scipy.stats as ss
import statistics as stats
import seaborn as sns 
from pandas.plotting import autocorrelation_plot
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image  
import pydotplus
from sklearn.model_selection import  GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score, precision_score, confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

In [2]:
db = pd.read_csv('/content/Base datos COVID-Linfos Actualizazda.xlsx - Valores seleccionados.csv')
#db = pd.read_excel('Base datos COVID-Linfos Actualizazda.xlsx') #no quites esta línea, coméntala cuando estés trabajando en tus datos

db.head(8)

,Número,Repetición NHC,Código Externo,FALLECE,UCI,Día del Año,19nCv,Día de POSITIVO,Días respecto a Positivo,Fecha,...,ESSYS,ASLYP,ASLYA,RELPL,RELYP,RELYA,NEUGI,NEURI,ASLPL,19nCv.1
0,221.0,4,111561,NaN,NaN,90,NaN,NaN,NaN,30/03/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,319.0,13,102441,NaN,NaN,100,NaN,NaN,NaN,9/04/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,368.0,16,102441,NaN,NaN,104,NaN,NaN,NaN,13/04/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1759.0,9,104851,NaN,NaN,238,NaN,NaN,NaN,25/08/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,265277.0,7,11258,NaN,NaN,91,NaN,NaN,NaN,31/03/20,...,NaN,0,0,"5,7","2,3","0,13","154,8","50,2",0,NaN
5,325932.0,11,115555,NaN,NaN,98,NaN,NaN,NaN,7/04/20,...,NaN,0,0,"6,6","2,1","0,15","147,1","47,8",0,NaN
6,366742.0,5,111520,NaN,NaN,83,NaN,NaN,NaN,23/03/20,...,NaN,0,0,"5,4","1,8","0,13","152,6","49,9",0,NaN
7,374616.0,6,116651,NaN,NaN,97,NaN,NaN,NaN,6/04/20,...,NaN,0,0,"11,5",5,"0,25",150,"52,7",0,NaN


### 1.1 Eliminación de columnas y filas (consultado con Javier)

Tras una visualización general y una charla con los sanitarios que han transferido la base de datos, se llega a la conclusión de que hay que hacer una limpieza de la misma. Hay columnas que su significado y valores no tienen ninguna relevancia para el estudio que se va a llevar a cabo.

In [3]:
db1=db.drop(["Repetición NHC","Día del Año","Día de POSITIVO","Días respecto a Positivo","GSRH","GRHPC","GRHRN","GRHCE","FRSP","COMHE","AVSBL","AVSPL","AVSRJ","AVNRB","ESSYS"],axis=1) #axis=1 ya que lo que elimino son columnas.


A continuación elimino las filas que no aportan información.

In [4]:
db1= db1.dropna(thresh=9) #Mantiene solo las filas con al menos 9 valores que no sean NA. Esto me sirve ya que quiero eliminar aquellas filas
#que tengan NaN, es decir 19nCv y todos los parámetros del hemograma.
db1.shape

(320, 122)

Además teniendo en cuenta la informacion proporcionada por parte del hospital, elimino aquellas variables que son redundantes. Estas variables no aportan información adicional al futuro modelo de predicción.

In [5]:
db1=db1.drop(["NE_R","NEU_a","NEU_p","LY_R","LIN_a","LIN_p","MO_R","EO_R","BA_R"],axis=1) #axis=1 ya que lo que elimino son columnas.

### 1.2 Eliminación de columnas con alta correlación

Tras comprobar las variables con alta correlación en el dendrograma se llega a la conclusión junto con el médico Javier, de eliminar una serie de variables por ser redundantes. Los grupos de parámetros sobre los que se discute son: (ERBLp, ERBL) (NEURI, NESFL) (NEUGI, NESSC) (PLTI, PLT) (WBC-N, LEU, TNC-N, WBC-D):

In [6]:
db1=db1.drop(["ERBLp","NESFL","NESSC","PLT","WBC-N","TNC-N","WBC-D"],axis=1)

### 1.3 Agrupación de pacientes por gravedad


- UCI=1 y FALLECE=1 --> Muy grave
- UCI=0 y FALLLECE=1 --> Muy grave

- UCI=1 y FALLECE=0 --> Grave
- UCI=0 y FALLECE=0 --> Sano

In [7]:
data_all = db1[(db1['UCI'] == 1.0) & (db1['FALLECE'] == 1.0)]
data_death = db1[(db1['UCI'] == 0.0) & (db1['FALLECE'] == 1.0)]
data_regular = db1[(db1['UCI'] == 1.0) & (db1['FALLECE'] == 0.0)]
data_health = db1[(db1['UCI'] == 0.0) & (db1['FALLECE'] == 0.0)]

### 1.4 Tipo de cada columna

In [8]:
db1.info(verbose=True, null_counts=True) #Hay un total de 342 filas

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 4 to 341
Data columns (total 106 columns):
 #    Column             Non-Null Count  Dtype  
---   ------             --------------  -----  
 0    Número             319 non-null    float64
 1    Código Externo     320 non-null    int64  
 2    FALLECE            96 non-null     float64
 3    UCI                96 non-null     float64
 4    19nCv              105 non-null    object 
 5    Fecha              320 non-null    object 
 6    Edad del Paciente  320 non-null    object 
 7    Sexo del Paciente  320 non-null    object 
 8    LEU                284 non-null    object 
 9    NEU                284 non-null    object 
 10   NEUp               284 non-null    object 
 11   LIN                284 non-null    object 
 12   LINp               284 non-null    object 
 13   MON                284 non-null    object 
 14   MONp               284 non-null    object 
 15   EOS                284 non-null    object 
 16   EOSp  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [9]:
#Cambio de comas por puntos
db1 = db1.replace(',','.', regex=True)

#Reemplazo esos caracteres por NaN
db1=db1.replace("----",np.nan)

# Función que elimina la palabra años
def only_digits (txt):
  return "".join(c for c in txt if c.isdigit())

db1["Edad del Paciente"] = db1["Edad del Paciente"].map(only_digits)

## MUY IMPORTANTE. Conversión de variables excepto 19nCv, Fecha, Sexo del Paciente y 19nCv.1 
# Una vez hecho el cambio de comas por puntos y haber introducido NaN por "----", este código me transforma los 'object' en 'numerics'
for i in db1:
  if not (i=="19nCv" or i=="Fecha" or i=="Sexo del Paciente" or i=="19nCv.1"):
    if db1[i].dtypes == object:
      db1[i] = pd.to_numeric(db1[i])

#Paso la variable "fecha" de tipo object a tipo datatime, para poder tratar con ella como una fecha.
db1["Fecha"]=pd.to_datetime(db1["Fecha"])

## 1.5 Intruducción de nuevas columnas, "Revision" y "Label"


In [10]:
Codigo_lista = db1['Código Externo'].tolist()

In [11]:
from collections import Counter

lista_pacientes = Counter(Codigo_lista).keys()

### INICIALIZO DATAFRAME DE SANOS
sanos_df = pd.DataFrame()

### INICIALIZO DATAFRAME DE NO SANOS
no_sanos_df = pd.DataFrame()

for i in lista_pacientes:  
      if i not in data_health.values:
        new_1 = db1.loc[db1["Código Externo"] == i]
        new_1 = new_1.sort_values('Fecha') #ordenar de la revision mas antigua a la mas reciente.
        new_1['Revision'] = range(1, 1+len(new_1)) #en la nueva columna "Revision" se van a añadiendo valores de manera incremental desde 1 teniendo 
                                          # en cuenta el número de citas de cada paciente.

        new_1 = new_1.assign(label=1) #Introduzco nueva columna con etiqueta 1 (NO SANO)

        no_sanos_df = no_sanos_df.append(new_1)

      else:
        new_2 = db1.loc[db1["Código Externo"] == i]
        new_2 = new_2.sort_values('Fecha')
        new_2['Revision'] = range(1, 1+len(new_2)) #en la nueva columna "Revision" se van a añadiendo valores de manera incremental desde 1 teniendo 
                                          # en cuenta el número de citas de cada paciente.

        new_2 = new_2.assign(label=0)#Introduzco nueva columna con etiqueta 0 (SANO)
      
        sanos_df = sanos_df.append(new_2)


# 2. Preprocesado de datos

### ¿Hasta que revisión se cuenta con un número adecuado de pacientes?

Se realiza el proyecto con un total de 4 revisiones. Se llega a esta conclusión de la siguiente forma; se dividen las variables de forma individual y se va representando cada uno de los valores de cada variable en cada revisión (teniendo así una visión de los valores de la variable para todos los pacientes en cada revisión a medida que pasa el tiempo). Esto se realiza con los pacientes sanos por un lado y los pacientes no sanos por el otro. Finalmente, se observa como con 4 revisiones se puede hacer un estudio amplio y completo de las variables. 

Además, de 98 pacientes, 44 pacientes presentan 4 revisiones, es decir el 44,89% de los pacientes presentes en la base de datos, porcentaje considerable que ayuda a tomar la decision de usar 4 revisiones.


In [12]:
db1 = pd.concat([sanos_df, no_sanos_df])
db1.shape

print(db1.columns)

Index(['Número', 'Código Externo', 'FALLECE', 'UCI', '19nCv', 'Fecha',
       'Edad del Paciente', 'Sexo del Paciente', 'LEU', 'NEU',
       ...
       'ASLYA', 'RELPL', 'RELYP', 'RELYA', 'NEUGI', 'NEURI', 'ASLPL',
       '19nCv.1', 'Revision', 'label'],
      dtype='object', length=108)


## 2.1 Identificacion de valores perdidos

Junto con el estudio del tipo de variables, es básico conocer el número de observaciones disponibles y si todas ellas están completas. Los valores perdidos son muy importantes a la hora de crear modelos, la mayoría de algoritmos no aceptan observaciones incompletas o bien se ven muy influenciados por ellas.

In [13]:
# Columnas con missing values

for index,value in enumerate (db1.isnull().sum()):
  if value !=0:
    print(db1.columns[index],value)

Número 1
FALLECE 224
UCI 224
19nCv 215
LEU 36
NEU 36
NEUp 36
LIN 36
LINp 36
MON 36
MONp 36
EOS 36
EOSp 36
BAS 36
BASp 36
IG 36
IGp 36
ERIT 36
HGB 36
HCT 36
VCM 36
HCM 36
CHCM 36
RDW 36
ERBL 36
RET 311
RETp 311
FRI 311
RETHE 311
PLTI 38
PLTF 316
PLTO 311
VPM 43
IPF 316
MNEU 313
MLIN 313
MMON 313
MBAS 313
MEOS 313
MCAS 313
MMET 317
MMIE 318
MPRO 318
CPLAS 318
MBLS 318
MERI 317
rNe/L 160
rPL/L 160
HIPER 311
HYPO 311
MACR 320
MICRO 320
FRAGM 311
MacR 36
MicR 36
RBCO 320
MCHCO 311
BA-D# 36
NEFSC 36
BA-D% 36
BA-N# 36
BA-N% 36
D-HE 311
FRCA 311
HFLCA 36
HFLCP 36
HIPF 316
IPFA 316
IRF-Y 311
LY-WX 36
LY-WY 36
LY-WZ 36
LY-X 36
LY-Y 36
LY-Z 36
MO-WX 36
MO-WY 36
MO-WZ 36
MO-X 36
MO-Y 36
MO-Z 36
NE-WX 36
NE-WY 36
NE-WZ 36
RBCHE 311
RBC-Y 311
RTNC 311
RUPP 311
RET-Y 311
RPI 311
TNC-D 320
TNC-P 320
WBC-P 320
ASLYP 42
ASLYA 42
RELPL 42
RELYP 42
RELYA 42
NEUGI 36
NEURI 36
ASLPL 42
19nCv.1 269


In [14]:
#Quitar todas las columnas con mas de 310 NaN

#320-310 = 10. Las columnas con 311 NaN tienen 9 valores no NaN. Por lo que si me quedo con aquellas columnas que tengan como mínimo 10 valores
# no NaN, elimino todas las columnas con 333 NaN o más.
db1=db1.dropna(thresh=10,axis=1) 

db1=db1.drop(["Número","19nCv","19nCv.1", "FALLECE", "UCI", "Fecha"],axis=1)


print(db1.columns)

Index(['Código Externo', 'Edad del Paciente', 'Sexo del Paciente', 'LEU',
       'NEU', 'NEUp', 'LIN', 'LINp', 'MON', 'MONp', 'EOS', 'EOSp', 'BAS',
       'BASp', 'IG', 'IGp', 'ERIT', 'HGB', 'HCT', 'VCM', 'HCM', 'CHCM', 'RDW',
       'ERBL', 'PLTI', 'VPM', 'rNe/L', 'rPL/L', 'MacR', 'MicR', 'BA-D#',
       'NEFSC', 'BA-D%', 'BA-N#', 'BA-N%', 'HFLCA', 'HFLCP', 'LY-WX', 'LY-WY',
       'LY-WZ', 'LY-X', 'LY-Y', 'LY-Z', 'MO-WX', 'MO-WY', 'MO-WZ', 'MO-X',
       'MO-Y', 'MO-Z', 'NE-WX', 'NE-WY', 'NE-WZ', 'ASLYP', 'ASLYA', 'RELPL',
       'RELYP', 'RELYA', 'NEUGI', 'NEURI', 'ASLPL', 'Revision', 'label'],
      dtype='object')


A continuación se hace un tratamiento de la variable categorica, para facilitar el trabajo con ella.

In [15]:
# Se convierte la variable Sexo del paciente a variable categórica

db1['Sexo del Paciente'] = db1['Sexo del Paciente'].astype('category').cat.codes # 0 --> Femenino, 1--> Masculino

In [16]:
db1

,Código Externo,Edad del Paciente,Sexo del Paciente,LEU,NEU,NEUp,LIN,LINp,MON,MONp,...,ASLYP,ASLYA,RELPL,RELYP,RELYA,NEUGI,NEURI,ASLPL,Revision,label
122,11258,65,1,5.45,3.74,68.7,1.07,19.6,0.48,8.8,...,1.5,0.08,15.9,3.1,0.17,155.1,53.9,7.5,1,0
152,11258,65,1,5.01,2.68,53.5,1.42,28.3,0.58,11.6,...,0.0,0.00,16.2,4.6,0.23,149.9,49.8,0.0,2,0
4,11258,65,1,5.57,2.55,45.8,2.27,40.8,0.47,8.4,...,0.0,0.00,5.7,2.3,0.13,154.8,50.2,0.0,3,0
180,115555,69,1,10.38,8.65,83.3,0.88,8.5,0.78,7.5,...,0.4,0.04,12.5,1.1,0.11,143.9,48.3,4.5,1,0
124,115555,69,1,7.11,3.61,50.8,2.85,40.1,0.62,8.7,...,0.0,0.00,3.2,1.3,0.09,145.3,48.4,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,114819,69,0,13.67,8.29,60.7,2.08,15.2,2.54,18.6,...,1.4,0.19,13.9,2.1,0.29,131.2,35.4,9.1,4,1
328,116453,66,1,11.14,9.94,89.3,0.64,5.7,0.18,1.6,...,0.5,0.06,17.2,1.0,0.11,153.2,49.9,9.4,1,1
332,116453,66,1,15.00,11.28,75.1,1.50,10.0,0.97,6.5,...,0.3,0.04,7.3,0.7,0.11,160.2,48.2,2.7,2,1
336,116453,66,1,11.49,10.30,89.6,0.49,4.3,0.45,3.9,...,0.0,0.00,10.2,0.4,0.05,160.0,50.6,0.0,3,1


## 2.2 XGBoost

Comienzo seleccionando únicamente los parámetros que elgió el método XGBoost (Revisar los parámetros seleccionados por los métodos cuando Javier me responda a la eliminación de features muy correlacionadas)

In [17]:
db1_FS_XGB = db1[['Código Externo', 'MicR','LINp','HGB','HCM','NEUp','HFLCA','RELPL','ERIT','IGp','Revision', "label"]]

In [18]:
db1_FS_XGB

,Código Externo,MicR,LINp,HGB,HCM,NEUp,HFLCA,RELPL,ERIT,IGp,Revision,label
122,11258,2.0,19.6,14.5,28.8,68.7,0.08,15.9,5.03,0.7,1,0
152,11258,1.8,28.3,13.6,28.0,53.5,0.05,16.2,4.86,1.6,2,0
4,11258,1.6,40.8,15.7,29.6,45.8,0.00,5.7,5.31,0.2,3,0
180,115555,2.2,8.5,13.2,28.8,83.3,0.04,12.5,4.59,0.6,1,0
124,115555,2.4,40.1,16.2,29.3,50.8,0.02,3.2,5.52,0.3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
321,114819,4.1,15.2,14.5,28.3,60.7,0.19,13.9,5.12,4.8,4,1
328,116453,0.9,5.7,14.9,31.2,89.3,0.06,17.2,4.78,2.0,1,1
332,116453,0.9,10.0,13.7,30.6,75.1,0.04,7.3,4.47,7.1,2,1
336,116453,0.7,4.3,12.4,31.6,89.6,0.01,10.2,3.93,1.6,3,1


Ahora debo comenzar con la división de train y test. Hay que tener en cuenta, que a la hora de dividir no me convendría tener la revisión 1 y 3 de un paciente en train y la revisión 2 del mismo paciente en test, por lo que eso debo de tenerlo MUY EN CUENTA

Por lo tanto, la división hay que hacerla por código externo.

Los pasos a seguir para llevar a cabo dicha división serán:

1. Extraer el código externo

2. Eliminar las duplicidades consiguiendo de esa manera un vector de valores únicos de la columna "Código Externo"

3. Shuffle del vector creado

4. Teniendo en cuenta que para train me quedo con el 80% de los datos y para test con el 20% restante, de los 98 pacientes que tengo, me quedaré con 78 para train y 20 para test

In [19]:
# Codigo_lista muestra todos los códigos externos y ha sido creada a partir de convertir a lista la columna 
# "Código Externo"

cd_unique = np.unique(Codigo_lista)
print("La lista de códigos externos únicos es:",cd_unique)

La lista de códigos externos únicos es: [  1164   1214  10074  10092  11197  11258  11372  12016  13299  13397
 101480 101609 101660 102441 104431 104798 104849 104851 105140 105582
 105639 106066 107746 107747 108527 108894 108897 109036 109115 109129
 109339 109887 110011 110435 110607 111310 111339 111520 111561 111786
 112135 112511 112823 112894 114183 114495 114819 115555 115784 115847
 116005 116033 116159 116453 116651 116806 117212 117398 117462 118023
 118401 119606 119737 119766 120418 121289 121358 122654 122958 123156
 123456 123676 123724 124738 124850 126455 126638 127383 128124 128148
 128864 130481 130937 131267 131394 132015 132432 133617 133978 134492
 134855 135062 135217 136647 136922 137069 137304 138076]


## 2.3 Separación en train y test

In [20]:
import random

#Desordeno de manera aleatoria la lista de códigos externos
random.shuffle(cd_unique)

#Hago la división en train y test, quedándome con los 78 primeros valores de la lista para train y los 20
#valores restantes para test

cd_train = cd_unique[0:78]
cd_test = cd_unique[78:98]

Debo crear un dataframe donde estén todas las revisiones de los primeros 78 códigos externos seleccionados anteriormente. A partir de este dataframe sacaré un x_train y un y_train que será "label".

In [ ]:
lista_codigos = Counter(cd_train).keys()

### INICIALIZO DATAFRAME DE TRAINING
training = pd.DataFrame()

for i in lista_codigos:  
      new_1 = db1.loc[db1["Código Externo"] == i]
      new_1 = new_1.sort_values('Fecha') #ordenar de la revision mas antigua a la mas reciente.
      new_1['Revision'] = range(1, 1+len(new_1)) #en la nueva columna "Revision" se van a añadiendo valores de manera incremental desde 1 teniendo 
                                          # en cuenta el número de citas de cada paciente.

      new_1 = new_1.assign(label=1) #Introduzco nueva columna con etiqueta 1 (NO SANO)

      no_sanos_df = no_sanos_df.append(new_1)